In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import molpy as mp

In [3]:
ff = mp.ForceField()
atomstyle = ff.def_atom_style('full')

O = atomstyle.def_atom_type(1, mass=15.999)
H = atomstyle.def_atom_type(2, mass=1.008)

In [8]:
bondstyle = ff.def_bond_style('harmonic', mix=None)

bondstyle.def_bond_type(id=0, idx_i=0, idx_j=1, k0=1, r0=1)
bondstyle.def_bond_type(id=1, idx_i=1, idx_j=2, k0=2, r0=2)

k0 = bondstyle.get_bondtype_params('k0')
assert k0.shape == (3, 3)
assert k0[0, 0] == 0

In [22]:
pairstyle = ff.def_pairstyle('lj/cut', mix='arithmetic')
pairstyle.def_pairtype(1, 0, 0, epsilon=3, sigma=3)
pairstyle.def_pairtype(2, 1, 1, epsilon=1, sigma=1)
pairstyle.def_pairtype(3, 2, 2, epsilon=2, sigma=2)

epsilon = pairstyle.get_pairtype_params('epsilon')
print(epsilon)
assert epsilon.shape == (3, 3)
assert epsilon[0, 0] == 3
assert epsilon[0, 1] == 2

[[3.  2.  2.5]
 [2.  1.  1.5]
 [2.5 1.5 2. ]]


In [ ]:
pair_pot = mpot.classical.pair.LJ126(eps=epsilon, sig=sigma, cutoff=pairstyle['cutoff'])

inputs = dict(
    dij = torch.randn((5, )),
    atomtype = torch.randint(0, 3, (5,)),
)

pair_energy = pair_pot(inputs)

In [ ]:
bond_pot = mpot.classical.bond.Harmonic(k0=bondstyle['k0'], r0=bondstyle['r0'])
inputs.update(dict(
    bond_i = torch.randint(0, 3, (5,)),
    bond_j = torch.randint(0, 3, (5,)),
))
bond_energy = bond_pot(inputs)